In [1]:
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2.service_account import Credentials

In [3]:
# BIGQUERY_CREDENTIALS_FILE_PATH = r"D:\data_engineer\TIR_group2\TIR101_Group2\secrets\harry_GCS_BigQuery_write_cred.json"
BIGQUERY_CREDENTIALS_FILE_PATH = r"C:\TIR101_Group2\secrets\harry_GCS_BigQuery_write_cred.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = BIGQUERY_CREDENTIALS_FILE_PATH
BQ_CLIENT = bigquery.Client()

In [12]:
def create_time_table(client:bigquery.Client,dataset_name:str,table_name:str,start_date:str,end_date:str):
    query_job = client.query(
        f""" 
        CREATE OR REPLACE TABLE `{dataset_name}.{table_name}`
        (   `date` DATE,
            `year` INT,
            `month` INT,
            `day` INT ,
            `day_of_week` INT,
            `week_of_year` INT,
            `month_name` STRING,
            `day_of_week_name` STRING
        );
        """
    )
    query_job.result()
    print(f"create time table with schema in {dataset_name}.{table_name}")
    query_job = client.query(
        f""" 
        INSERT INTO `{dataset_name}.{table_name}`(
            date,year,month,day,day_of_week,week_of_year,month_name,day_of_week_name
        )
        SELECT
            `date`,
            EXTRACT(YEAR FROM `date`),
            EXTRACT(MONTH FROM `date`),
            EXTRACT(DAY FROM `date`),
            EXTRACT(DAYOFWEEK FROM `date`),
            EXTRACT(WEEK FROM `date`),
            FORMAT_DATE('%B',`date`),
            FORMAT_DATE('%A',`date`)
        FROM 
            UNNEST(GENERATE_DATE_ARRAY('{start_date}', '{end_date}')) AS `date`;
        """
    )
    query_job.result()
    print(f"insert data from '{start_date}' to '{end_date}'")

In [13]:
create_time_table(
    client=BQ_CLIENT,
    dataset_name="ANDY_ETL_DIM",
    table_name="DIM_time_table",
    start_date="2020-01-01",
    end_date="2030-01-01"
)

create time table with schema in ANDY_ETL_DIM.DIM_time_table
insert data from '2020-01-01' to '2030-01-01'


In [16]:
def download_time_table(client:bigquery.Client,dataset_name:str,table_name:str):
    query_job = client.query(
    f""" 
    SELECT * FROM  `{dataset_name}.{table_name}`
    WHERE `year`=2024 AND `month`=5
    ORDER BY `date`;
    """
    )
    return(query_job.to_dataframe())
df = download_time_table(
    client=BQ_CLIENT,
    dataset_name="ANDY_ETL_DIM",
    table_name="DIM_time_table",
)

In [17]:
df

,date,year,month,day,day_of_week,week_of_year,month_name,day_of_week_name
0,2024-05-01,2024,5,1,4,17,May,Wednesday
1,2024-05-02,2024,5,2,5,17,May,Thursday
2,2024-05-03,2024,5,3,6,17,May,Friday
3,2024-05-04,2024,5,4,7,17,May,Saturday
4,2024-05-05,2024,5,5,1,18,May,Sunday
5,2024-05-06,2024,5,6,2,18,May,Monday
6,2024-05-07,2024,5,7,3,18,May,Tuesday
7,2024-05-08,2024,5,8,4,18,May,Wednesday
8,2024-05-09,2024,5,9,5,18,May,Thursday
9,2024-05-10,2024,5,10,6,18,May,Friday
